In [26]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [27]:
%matplotlib inline

In [2]:
def read_ncdf(file,vars=[]):
    """ Reads a netCDF 3 file and returns a dict with its variables
    
    Parameters
    ----------
    file : string
        The netCDF file to be read
    vars : string list, optional
        List of variables to be read from file

    Returns
    -------
    d : A python dict containing the files variables

    Examples
    --------
    >>> d=read_ncdf('test.cdf')
    >>> d1=read_ncdf('test.cdf',vars=['var1'])
    >>> d2=read_ncdf('test.cdf',vars=['var1','var2'])

    """
    from scipy.io import netcdf

    try:
        f=netcdf.netcdf_file(file,'r',mmap=False)
    except IOError:
        print('Error: Cannot open file'+file)
        return 0

    if vars == []: vars=f.variables.keys()
    d=dict((v,f.variables[v].data) for v in vars)
    f.close()
    return d

In [254]:
def gaussian(xdata,muE,sigE,muP,sigP,norm):
    E = xdata[0]
    P = xdata[1]
    gaussian = np.exp(-(((E-muE)/sigE)**2 + (((P-muP)/sigP)**2) ))
    gaussian *= norm
    return gaussian

In [214]:
def bimax(xdata,Eperp,Epara,norm):
    E = xdata[0,:]
    P = xdata[1,:]
    bimax = (1.0/np.sqrt(np.pi))*(1.0/Eperp)*(1.0/np.sqrt(Epara))*np.sqrt(E)*np.exp(-(P*P*E/Epara))*np.exp(-((1-P*P)*E/Eperp))
    bimax *= norm
    return bimax

In [238]:
def fit_bimax():
    files = ["data/"+f+"_sampled_bimax.cdf" for f in ["tsvd","tik0","tik1","mf","me"]]
    print("Epara = 3 keV")
    print("Eperp = 20 keV")
    print("Norm = 2.43e12\n")
    for f in files:
        d = read_ncdf(f)
        error_level = d["error_level"]
        Egrid,Pgrid = np.meshgrid(d["energy"],d["pitch"])
        Egrid = Egrid.T
        Pgrid = Pgrid.T
        E = np.ravel(Egrid)
        P = np.ravel(Pgrid)
        xdata = np.zeros((2,400))
        xdata[0,:] = E
        xdata[1,:] = P
        print(f)
        print("Error Level\tEperp\t\tEpara\t\tNorm")
        for i,e in enumerate(error_level):
            ydata = d["x_mean"][i,:]
            err = np.max(np.sqrt(d["x_var"][i,:]),1e-30)
            p, pcov = curve_fit(bimax,xdata,ydata,sigma = err, p0=[20.0,3.0,2.43e12])
            perr = np.sqrt(np.diag(pcov))
            print("{err:4.2f}\t\t{eperp:5.2f} +/- {eperperr:3.2f}\t{epara:4.2f} +/- {eparaerr:4.2f}\t{norm:6.2E} +/- {normerr:6.2E}".format(err=e,eperp=p[0],eperperr=perr[0],epara=p[1],eparaerr=perr[1],norm=p[2],normerr=perr[2]))

In [239]:
fit_bimax()

Epara = 3 keV
Eperp = 20 keV
Norm = 2.43e12

data/tsvd_sampled_bimax.cdf
Error Level	Eperp		Epara		Norm
0.05		24.03 +/- 0.50	3.97 +/- 0.11	2.12E+12 +/- 2.86E+10
0.10		24.27 +/- 0.53	4.01 +/- 0.11	2.11E+12 +/- 2.98E+10
0.15		24.42 +/- 0.55	4.09 +/- 0.12	2.10E+12 +/- 3.06E+10
0.20		24.32 +/- 0.56	4.19 +/- 0.12	2.11E+12 +/- 3.11E+10
0.25		24.55 +/- 0.58	4.27 +/- 0.13	2.10E+12 +/- 3.17E+10
0.30		24.61 +/- 0.63	4.60 +/- 0.15	2.09E+12 +/- 3.39E+10
0.35		24.55 +/- 0.64	4.69 +/- 0.15	2.10E+12 +/- 3.47E+10
0.40		24.44 +/- 0.67	4.90 +/- 0.17	2.08E+12 +/- 3.59E+10
0.45		24.68 +/- 0.72	5.01 +/- 0.18	2.05E+12 +/- 3.77E+10
0.50		24.36 +/- 0.73	5.12 +/- 0.19	2.06E+12 +/- 3.86E+10
0.55		24.26 +/- 0.74	5.18 +/- 0.20	2.06E+12 +/- 3.95E+10
0.60		24.16 +/- 0.76	5.33 +/- 0.21	2.05E+12 +/- 4.04E+10
0.65		24.36 +/- 0.82	5.59 +/- 0.23	2.00E+12 +/- 4.16E+10
0.70		23.75 +/- 0.78	5.36 +/- 0.21	2.03E+12 +/- 4.18E+10
0.75		23.71 +/- 0.81	5.52 +/- 0.23	2.01E+12 +/- 4.33E+10
0.80		23.12 +/- 0.80	5.63 +/- 0.23	2.01E+

In [266]:
def fit_gaussian():
    files = ["data/"+f+"_sampled_gaussian.cdf" for f in ["tsvd","tik0","tik1","mf","me"]]
    print("muE = 50 keV")
    print("sigE = 10 keV")
    print("muP = 0")
    print("sigE = 0.25")
    print("Norm = 1.27e11\n")
    for f in files:
        d = read_ncdf(f)
        error_level = d["error_level"]
        Egrid,Pgrid = np.meshgrid(d["energy"],d["pitch"])
        Egrid = Egrid.T
        Pgrid = Pgrid.T
        E = np.ravel(Egrid)
        P = np.ravel(Pgrid)
        xdata = np.zeros((2,400))
        xdata[0,:] = E
        xdata[1,:] = P
        print(f)
        print("Error Level\tmuE\t\tsigE\t\t muP\t\t\tsigP\t\t\tNorm")
        for i,e in enumerate(error_level):
            ydata = d["x_mean"][i,:]
            err = np.max(np.sqrt(d["x_var"][i,:]),1e-30)
            p, pcov = curve_fit(gaussian,xdata,ydata,sigma = err, p0=[50.0,10.0,0.0,0.25,1.27e11])
            perr = np.sqrt(np.diag(pcov))
            print("""{err:4.2f}\t\t{muE:5.2f} +/- {muEerr:4.2f}\t{sigE:5.2f} +/- {sigEerr:4.2f}\t{muP:4.2E} +/- {muPerr:4.2E}\t{sigP:4.2E} +/- {sigPerr:4.2E}\t{norm:6.2E} +/- {normerr:6.2E}
                  """.format(err=e,muE=p[0],muEerr=perr[0],sigE=p[1],sigEerr=perr[1],muP=p[2],muPerr=perr[2],sigP=p[3],sigPerr=perr[3],norm=p[4],normerr=perr[4]))

In [267]:
fit_gaussian()

muE = 50 keV
sigE = 10 keV
muP = 0
sigE = 0.25
Norm = 1.27e11

data/tsvd_sampled_gaussian.cdf
Error Level	muE		sigE		 muP			sigP			Norm
0.05		50.08 +/- 0.13	12.29 +/- 0.18	-6.77E-04 +/- 2.87E-03	2.81E-01 +/- 4.07E-03	1.00E+11 +/- 1.45E+09
                  
0.10		49.97 +/- 0.13	13.03 +/- 0.18	-4.98E-03 +/- 2.89E-03	2.93E-01 +/- 4.08E-03	9.31E+10 +/- 1.30E+09
                  
0.15		49.89 +/- 0.16	14.22 +/- 0.22	-7.62E-03 +/- 3.39E-03	3.08E-01 +/- 4.80E-03	8.36E+10 +/- 1.30E+09
                  
0.20		49.95 +/- 0.19	15.05 +/- 0.27	-5.24E-04 +/- 4.14E-03	3.26E-01 +/- 5.86E-03	7.62E+10 +/- 1.37E+09
                  
0.25		49.90 +/- 0.20	15.26 +/- 0.29	2.85E-03 +/- 4.35E-03	3.28E-01 +/- 6.15E-03	7.46E+10 +/- 1.40E+09
                  
0.30		49.91 +/- 0.21	15.22 +/- 0.29	3.23E-03 +/- 4.46E-03	3.28E-01 +/- 6.31E-03	7.46E+10 +/- 1.43E+09
                  
0.35		49.75 +/- 0.21	15.25 +/- 0.30	2.42E-03 +/- 4.57E-03	3.28E-01 +/- 6.47E-03	7.40E+10 +/- 1.46E+09
                  
0.40		49.72 +